# Anaconda3下でのInstall

condaではtensorboard実行に問題が出たので一旦これで。

```
$ pip install -i https://pypi.anaconda.org/jjhelmus/simple tensorflow
```


# Installを確認する

In [3]:
import tensorflow as tf

hello = tf.constant('Hi, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hi, TensorFlow!'


# TensorFlowを簡単な計算で理解する

> [TensorFlowを算数で理解する - Qiita](http://qiita.com/icoxfog417/items/fb5c24e35a849f8e2c5d)

### スカラ量で計算してみる

- 計算する数式
$$ 
y = x^2 + b
$$


- flowを図示すると

```
0.5 --> x--[square]--[add]--> result
                      |
3.0 --> b-------------+

```


In [11]:
def x2_plus_b(x, b):
    _x = tf.constant(x)
    _b = tf.constant(b)
    result = tf.square(_x)
    result = tf.add(result, _b)
    return result

sess = tf.Session()
print(sess.run([x2_plus_b(.5,3.)]))

[3.25]


### TensorBoardによる可視化

- 実行後に、コマンドラインから下記を実行
 
 ```
$ tensorboard --logdir=/path/to/log-directory
 ```

- `http://0.0.0.0:6006`にアクセスする

- `GRAPH`に実行済フローが複数表示される

> ポイントとしては、`tf.scalar_summary`で計算した値をsummaryしておく点です。こうして計算したsummaryを、`tf.train.SummaryWriter`で書き出していきます。 http://qiita.com/icoxfog417/items/fb5c24e35a849f8e2c5d

In [13]:
import tensorflow as tf

def monitor_calculation(x, b):
    title = "b = {0}".format(b)
    c = x2_plus_b(float(x), float(b))
    s = tf.scalar_summary(title, c)
    m = tf.merge_summary([s])  # if you are using some summaries, merge them
    return m

with tf.Session() as sess:
    writer = tf.train.SummaryWriter("log", graph_def=sess.graph_def)    
    xaxis = range(-10, 12)

    for b in range(3):
        for x in xaxis:
            summary_str = sess.run(monitor_calculation(x, b))
            writer.add_summary(summary_str, x)

### スカラ量で計算する

> https://www.tensorflow.org/versions/master/get_started/index.html

上記コードはPython2ベースのため、`xrange()` -> `range()`に修正。

> 参考 http://stackoverflow.com/questions/17192158/nameerror-global-name-xrange-is-not-defined-in-python-3

- やっていることは回帰分析

> 「y = 0.1 x + 0.3」という数式を使って生成したxとyを学習データとして、y = W x + b という数式のWとbを最適化していく http://qiita.com/MATS_ELB/items/dfc50149d52e47e5a07b


In [20]:
import tensorflow as tf
import numpy as np

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but Tensorflow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

# Learns best fit is W: [0.1], b: [0.3]

0 [-0.07575795] [ 0.5411855]
20 [ 0.02578249] [ 0.33941314]
40 [ 0.07675866] [ 0.31234229]
60 [ 0.09272193] [ 0.30386502]
80 [ 0.09772088] [ 0.30121034]
100 [ 0.09928628] [ 0.30037904]
120 [ 0.0997765] [ 0.30011871]
140 [ 0.09993] [ 0.30003718]
160 [ 0.09997807] [ 0.30001166]
180 [ 0.09999313] [ 0.30000365]
200 [ 0.09999785] [ 0.30000114]


### テンソルで計算する

> http://qiita.com/MATS_ELB/items/fec7f54de2dd18b043ae

In [28]:
# tfという名前で参照できるようにtensorflowを、npという名前で参照できるようにインポート
import tensorflow as tf
import numpy as np

# y_data = W_data * x_data + b_data というx_dataとy_dataの関係になるようにあらかじめ
# W_dataとb_dataの値を定める。機械学習が適切に行われるとWはこのW_dataに、bはこのb_dataに近づく
# なお、W_dataは２行２列のテンソル、b_dataは２行１列のテンソル
W_data = np.array([[0.1, 0], [0, 0.1]])
b_data = np.array([0.3, 0.3])

# 乱数生成を利用して0から1の間の数値を持つ２行１列の行列「X_data」を浮動小数として100個生成
x_data = np.random.rand(100, 2, 1).astype("float32")

# その後で、生成した100個のxに対して、行列版でのy=0.1x+0.3となるようなyを100個生成
y_data = W_data * x_data + b_data

# 上記で生成したxとy（共に２行１列のテンソル）の組を学習データとして用いる

# 機械学習で最適化するWとbを設定する。Wは２行２列のテンソル。bは２行１列のテンソル。
W = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0))
b = tf.Variable(tf.zeros([2]))
y = W * x_data + b

# 学習において、その時点での学習のダメ程度を表すlossを、学習データのyとその時点でのyの差の２乗と定義
# Wとbの最適化のアルゴリズムを最急降下法（勾配法）とし、その１回の最適化処理にoptimizerと名前を付ける
# 上記の最適化処理の繰り返しによりlossを最小化する処理をtrainと呼ぶことにする
loss = tf.reduce_mean(tf.square(y_data - y))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 学習を始める前にこのプログラムで使っている変数を全てリセットして空っぽにする
init = tf.initialize_all_variables()

# Launch the graph.（おきまりの文句）
sess = tf.Session()
sess.run(init)

# 学習を1000回行い、100回目ごとに画面に学習回数とWとbのその時点の値を表示する
for step in range(1001):
    sess.run(train)
    if step % 100 == 0:
        print(step, sess.run(W), sess.run(b))

# Learns best fit is W: [[0.1, 0], [0, 0.1]], b: [0.3, 0.3]

0 [[ 0.12049839  0.63580847]
 [ 0.7111249  -0.02813603]] [ 0.05483134  0.09086838]
100 [[ 0.15628858  0.04232659]
 [ 0.05392477  0.14043736]] [ 0.27025187  0.27766153]
200 [[ 0.10838222  0.00629473]
 [ 0.00802105  0.10602347]] [ 0.29557258  0.29667521]
300 [[ 0.10124753  0.00093688]
 [ 0.00119379  0.10089649]] [ 0.29934105  0.29950514]
400 [[ 0.10018568  0.00013946]
 [ 0.0001777   0.10013343]] [ 0.29990193  0.29992634]
500 [[  1.00027628e-01   2.07803114e-05]
 [  2.64537048e-05   1.00019872e-01]] [ 0.29998541  0.29998901]
600 [[  1.00004129e-01   3.12404495e-06]
 [  3.96192627e-06   1.00002974e-01]] [ 0.29999781  0.29999834]
700 [[  1.00000627e-01   5.01336388e-07]
 [  6.14700070e-07   1.00000471e-01]] [ 0.29999965  0.29999974]
800 [[  1.00000247e-01   2.22809277e-07]
 [  2.17059082e-07   1.00000232e-01]] [ 0.29999986  0.29999986]
900 [[  1.00000247e-01   2.22755844e-07]
 [  2.14493255e-07   1.00000232e-01]] [ 0.29999986  0.29999986]
1000 [[  1.00000247e-01   2.22767198e-07]
 [  2.1447

### 次は同内容のTensorboard可視化（エラーあり）

> http://qiita.com/MATS_ELB/items/fec7f54de2dd18b043ae

In [ ]:
# tfという名前で参照できるようにtensorflowを、npという名前で参照できるようにインポート
import tensorflow as tf
import numpy as np

# y_data = W_data * x_data + b_data というx_dataとy_dataの関係になるようにあらかじめ
# W_dataとb_dataの値を定める。機械学習が適切に行われるとWはこのW_dataに、bはこのb_dataに近づく
# なお、W_dataは２行２列のテンソル、b_dataは２行１列のテンソル
W_data = np.array([[0.1, 0], [0, 0.1]])
b_data = np.array([0.3, 0.3])

# 乱数生成を利用して0から1の間の数値を持つ２行１列の行列「X_data」を浮動小数として100個生成
x_data = np.random.rand(100, 2, 1).astype("float32")

# その後で、生成した100個のxに対して、行列版でのy=0.1x+0.3となるようなyを100個生成
y_data = W_data * x_data + b_data

# 上記で生成したxとy（共に２行１列のテンソル）の組を学習データとして用いる

# 機械学習で最適化するWとbを設定する。Wは２行２列のテンソル。bは２行１列のテンソル。
W = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0))
b = tf.Variable(tf.zeros([2]))
y = W * x_data + b

## TensorBoardへ表示するための変数を用意する（ヒストグラム用）
W_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
y_hist = tf.histogram_summary("y", y)

# 学習において、その時点での学習のダメ程度を表すlossを、学習データのyとその時点でのyの差の２乗と定義
# Wとbの最適化のアルゴリズムを最急降下法（勾配法）とし、その１回の最適化処理にoptimizerと名前を付ける
# 上記の最適化処理の繰り返しによりlossを最小化する処理をtrainと呼ぶことにする
loss = tf.reduce_mean(tf.square(y_data - y))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

## TensorBoardへloss（学習のダメ具合の指標として設定したスカラー値）を表示するための変数を用意する（イベント用）
loss_sum = tf.scalar_summary("loss", loss)

sess = tf.Session()

# 上記で用意した合計４つのTensorBoard描画用の変数を、TensorBoardが利用するSummaryデータとしてmerge（合体）する
# また、そのSummaryデータを書き込むSummaryWriterを用意し、書き込み先を'/tmp/tf_logs'ディレクトリに指定する
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter("/tmp/tf_logs", sess.graph_def)


# 学習を始める前にこのプログラムで使っている変数を全てリセットして空っぽにする
init = tf.initialize_all_variables()

# Launch the graph.（おきまりの文句）
sess.run(init)

# 学習を1000回行い、10回目ごとにSummaryデータを保存し、画面にWとbのその時点の値を表示する
for step in range(1001):
    if step % 10 == 0:
        result = sess.run([merged, loss])
        summary_str = result[0]
        writer.add_summary(summary_str, step)
        print(step, sess.run(W), sess.run(b))
    else:
        sess.run(train)

# Learns best fit is W: [[0.1, 0], [0, 0.1]], b: [0.3, 0.3]

# TensorFlow公式レポジトリを使う

### git cloneする

```
$ git clone https://github.com/tensorflow/tensorflow
```

`/tensorflow/tensorflow/examples`にtutorials等のsource codeあり。
